# Summary

This will cover calculating a baseline, calculating correction factors, 
and downscaling. The notebook assumed that you have run the prior notebooks
and have a tile directory with the following files in it:

  - `cru-input/cru-input-YYYY.nc`
  - `manifest.yml`
  - `worldclim.nc`

Start by loading libraries. 

In [ ]:
# For development...
# %load_ext autoreload
# %autoreload 2

from temds import tile, climate_variables


# Set up a log for the process

Our tools use an object to log and display various messages that may be created. There are several message priorities that
a logged message may have. All messages are saved, but only messages with the priority set in `verbose_levels` are printed to the screen\console. For this demo we will display all **INFO** messages, but if you are having issues it may be usefull to 
log all **DEBUG** messages. 

In [ ]:
from temds import logger
log = logger.Logger(verbose_levels=logger.INFO)

# Load the tile

This function will make an in memory `Tile` object with data from the directory passed to it.

In [ ]:
mytile = tile.Tile.tile_from_directory("working/03-tiles/H00_V08")
## set the logger for all tile objects
mytile.set_logger(log)
mytile

In [ ]:
for k,v in mytile.data.items():
  print(f"The tile has the key '{k}' to a variable of type {type(v)}")


In [ ]:
#start_year = mytile.data['cru_AnnualTimeSeries'].range()[0]
#end_year = mytile.data['cru_AnnualTimeSeries'].range()[-1]

start_year = 1970
end_year = start_year + 30
print(f"Start year: {start_year}, End year: {end_year}")

# Calculate the baseline

The docstring for `crujra.AnnualTimeSeries.create_climate_baseline()` is helpful:

In [ ]:
#print(crujra.AnnualTimeSeries.create_climate_baseline.__doc__)

Once the baseline is computed, it is now in memory as part of the `Tile` object. If you save the object you will additionally get a baseline.nc file and that should be added to the manifest.

In [ ]:
mytile.calculate_climate_baseline(start_year, start_year + 30, 'cru-baseline', 'cru-input')
mytile.data['cru-baseline']

# Correction Factors

Now that we have a climate baseline calcuated, we need to calculate correction
factors. The correction factors are handled slightly differently for different types
of variables (i.e. mean for some values, sum for others). Additionally the 
variables used are named slightly differently in the baseline and reference datasets.

So we start here by making a dictionary holding this info that we can pass to 
the correction factor function.

In [ ]:

# Calculate correction factors for the variables
mytile.calculate_correction_factors('cru-baseline','worldclim', climate_variables.DOWNSCALE_SAFE, 'cru-delta-cf')

Now our tile object has a new `correction_factors` key in the data dictionary:

In [ ]:
print(mytile.data.keys())
print()
print("Correction Factors:")
print(mytile.data['cru-delta-cf'])

# Downscaling

The downscaling resamples the data to a finer resolution and applies the 
correction factors.

In [ ]:
variables_ds = ['tair_avg', 'tair_max', 'tair_min', 'prec', 'nirr', 'wind', 'vapo', 'winddir']
mytile.downscale_timeseries('cru-downscaled', 'cru-input', 'cru-delta-cf', variables_ds, False)




Once we have the downscaling done, we can save the data for the tile.



In [ ]:
# Make sure the output directory exists
!mkdir -p working/04-downscaled-tiles/
mytile.save("working/04-downscaled-tiles", items=['cru-downscaled', 'cru-baseline', 'cru-delta-cf'], use_zlib=False)

In [ ]:
!ls working/04-downscaled-tiles/H00_V08/

# V ---- SCRATCH ---- V

In [ ]:
type(mytile.crs)

import pyproj

pyproj.CRS.from_user_input(mytile.crs).to_wkt()